## EM Algorithm - (b)

In [12]:
import numpy as np
from scipy.stats import multivariate_normal

import random

### Generate the training & testing dataset
* the first 500 data belong to class1, the last 500 data belong to class2 (both in the training and the testing set)

In [13]:
def generate_dataset_p1():
    m1 = np.array([1.25, 1.25])
    m2 = np.array([2.75, 2.75])
    m3 = np.array([2, 1.6])
    s1 = 0.1 * np.identity(2)
    s2 = 0.2 * np.identity(2)
    s3 = 0.3 * np.identity(2)
    
    norm1 = multivariate_normal(mean=m1, cov=s1)
    norm2 = multivariate_normal(mean=m2, cov=s2)
    norm3 = multivariate_normal(mean=m3, cov=s3)
    
    dataset = np.zeros((2, 500)) # each column corresponds to a data
    
    for i in range(500):
        decide_pdf = np.random.randint(0, 10)
        if 0 <= decide_pdf <= 3:
            dataset[:, i] = norm1.rvs()
        elif 4 <= decide_pdf <= 7:
            dataset[:, i] = norm2.rvs()
        else:
            dataset[:, i] = norm3.rvs()
    
    return dataset


def generate_dataset_p2():
    m1 = np.array([1.25, 2.75])
    m2 = np.array([2.75, 1.25])
    m3 = np.array([4, 6])
    s1 = 0.1 * np.identity(2)
    s2 = 0.2 * np.identity(2)
    s3 = 0.3 * np.identity(2)
    
    norm1 = multivariate_normal(mean=m1, cov=s1)
    norm2 = multivariate_normal(mean=m2, cov=s2)
    norm3 = multivariate_normal(mean=m3, cov=s3)
    
    dataset = np.zeros((2, 500)) # each column corresponds to a data
    
    for i in range(500):
        decide_pdf = np.random.randint(0, 10)
        if 0 <= decide_pdf <= 1:
            dataset[:, i] = norm1.rvs()
        elif 2 <= decide_pdf <= 4:
            dataset[:, i] = norm2.rvs()
        else:
            dataset[:, i] = norm3.rvs()
    
    return dataset

In [14]:
train = np.concatenate((generate_dataset_p1(), generate_dataset_p2()), axis=1)
test = np.concatenate((generate_dataset_p1(), generate_dataset_p2()), axis=1)

### Define functions of EM algorithm

In [15]:
def pdf(X, mu_l, cov_l):
    norm = multivariate_normal(mean=mu_l, cov=cov_l, allow_singular=True)
    return norm.pdf(X)


def e_step(dataset, alpha, mu, cov, expect, J):
    
    for l in range(J):
        numera = alpha[l] * pdf(np.transpose(dataset), mu[:, l], cov[:, :, l])

        denomi = 0
        for iter_l in range(J):
            denomi += alpha[iter_l] * pdf(np.transpose(dataset), mu[:, iter_l], cov[:, :, iter_l])

        expect[l, :] = numera / denomi
        
    return expect
            
        
def m_step(dataset, alpha, mu, cov, expect, J):
    
    # calculate new alpha
    alpha = np.sum(expect, axis=1) / dataset.shape[1]
    
    # calculate new mu
    for l in range(J):
        # expect[l]: probability density of each data of model l
        numera = np.sum(expect[l] * dataset, axis=1)
        denomi = np.sum(expect[l])
        mu[:, l] = numera / denomi
    
    # calculate new cov
    for l in range(J):
        
        numera = np.zeros((2, 2))
        for i in range(dataset.shape[1]):
            diff_vec = np.reshape(dataset[:, i] - mu[:, l], (2, 1))
            numera += expect[l, i] * diff_vec.dot(np.transpose(diff_vec))
        
        denomi = np.sum(expect[l])
        
        cov[:, :, l] = numera / denomi
        
    return alpha, mu, cov

### (b) - (1):   
estimate p1(x) and p2(x)

In [16]:
J = 3
    
#### define/initialize the variables to estimate
# parameters of class 1
alpha_c1 = np.zeros(J) # each element corresponds to a model
alpha_c1[:] = 1 / J

mu_c1 = np.random.rand(2, J) # each column corresponds to a model
mu_c1 *= J

cov_c1 = np.zeros((2, 2, J))
for i in range(J):
    cov_c1[:, :, i] = random.uniform(0, 1) * np.identity(2)

expect_c1 = np.zeros((J, 500)) # each column corresponds to a data

# parameters of class 2
alpha_c2 = np.zeros(J)
alpha_c2[:] = 1 / J

mu_c2 = np.random.rand(2, J)
mu_c2 *= J

cov_c2 = np.zeros((2, 2, J))
for i in range(J):
    cov_c2[:, :, i] = random.uniform(0, 1) * np.identity(2)

expect_c2 = np.zeros((J, 500))


#### train for class 1 with the training data which belong to class 1
for i in range(200):
    expect_c1 = e_step(train[:, :500], alpha_c1, mu_c1, cov_c1, expect_c1, J)
    alpha_c1, mu_c1, cov_c1 = m_step(train[:, :500], alpha_c1, mu_c1, cov_c1, expect_c1, J)
    if i == 199:
        print('parameters of the three models of the class 3 after training:')
        print('em iteration i =', i)
        for k in range(J):
            print('alpha_c1', k, '=', alpha_c1[k])
        for k in range(J):
            print('mu_c1', k, '=', mu_c1[:, k])
        for k in range(J):
            print('cov_c1', k, '=\n', cov_c1[:, :, k])

#### train for class 2 with the training data which belong to class 2
for i in range(200):
    expect_c2 = e_step(train[:, 500:], alpha_c2, mu_c2, cov_c2, expect_c2, J)
    alpha_c2, mu_c2, cov_c2 = m_step(train[:, 500:], alpha_c2, mu_c2, cov_c2, expect_c2, J)
    if i == 199:
        print('\n\nparameters of the three models of the class 2 after training:')
        print('em iteration i =', i)
        for k in range(J):
            print('alpha_c2', k, '=', alpha_c2[k])
        for k in range(J):
            print('mu_c2', k, '=', mu_c2[:, k])
        for k in range(J):
            print('cov_c2', k, '=\n', cov_c2[:, :, k])

parameters of the three models of the class 3 after training:
em iteration i = 199
alpha_c1 0 = 0.36715301760429214
alpha_c1 1 = 0.46691762390717373
alpha_c1 2 = 0.16592935848853416
mu_c1 0 = [1.64782109 1.40921055]
mu_c1 1 = [2.71052353 2.71197188]
mu_c1 2 = [1.18349451 1.24541188]
cov_c1 0 =
 [[0.27340815 0.01332311]
 [0.01332311 0.21315136]]
cov_c1 1 =
 [[ 0.19121626 -0.00193205]
 [-0.00193205  0.20183036]]
cov_c1 2 =
 [[0.06035753 0.0096439 ]
 [0.0096439  0.04109751]]


parameters of the three models of the class 2 after training:
em iteration i = 199
alpha_c2 0 = 0.29448508185713
alpha_c2 1 = 0.4840000002633138
alpha_c2 2 = 0.2215149178795562
mu_c2 0 = [2.73012045 1.22839123]
mu_c2 1 = [4.01305981 6.01019344]
mu_c2 2 = [1.32261869 2.72750678]
cov_c2 0 =
 [[0.19296048 0.0045372 ]
 [0.0045372  0.21092971]]
cov_c2 1 =
 [[0.30814225 0.00291631]
 [0.00291631 0.26067554]]
cov_c2 2 =
 [[ 0.10895812 -0.01332066]
 [-0.01332066  0.11666792]]


### (b) - (2)  
#### Train and test, loop by different J
* define/initialize the variables to estimate
* train
* test  
p.s. when J is large (about 10~20), the below code may enconter ValueError about inf and NaNs, I guess it's because of some extremely small values in alpha.

In [20]:
for J in range(1, 11):
    
    #### define/initialize the variables to estimate
    # parameters of class 1
    alpha_c1 = np.zeros(J) # each element corresponds to a model
    alpha_c1[:] = 1 / J

    mu_c1 = np.random.rand(2, J) # each column corresponds to a model
    mu_c1 *= J

    cov_c1 = np.zeros((2, 2, J))
    for i in range(J):
        cov_c1[:, :, i] = random.uniform(0, 1) * np.identity(2)

    expect_c1 = np.zeros((J, 500)) # each column corresponds to a data

    # parameters of class 2
    alpha_c2 = np.zeros(J)
    alpha_c2[:] = 1 / J

    mu_c2 = np.random.rand(2, J)
    mu_c2 *= J

    cov_c2 = np.zeros((2, 2, J))
    for i in range(J):
        cov_c2[:, :, i] = random.uniform(0, 1) * np.identity(2)

    expect_c2 = np.zeros((J, 500))

    
    #### train for class 1 with the training data which belong to class 1
    for i in range(200):
        expect_c1 = e_step(train[:, :500], alpha_c1, mu_c1, cov_c1, expect_c1, J)
        alpha_c1, mu_c1, cov_c1 = m_step(train[:, :500], alpha_c1, mu_c1, cov_c1, expect_c1, J)
#         if i == 199:
#             print('em iteration i =', i)
#             for k in range(J):
#                 print('alpha_c1', k, '=', alpha_c1[k])
#             for k in range(J):
#                 print('mu_c1', k, '=', mu_c1[:, k])
#             for k in range(J):
#                 print('cov_c1', k, '=\n', cov_c1[:, :, k])

    #### train for class 2 with the training data which belong to class 2
    for i in range(200):
        expect_c2 = e_step(train[:, 500:], alpha_c2, mu_c2, cov_c2, expect_c2, J)
        alpha_c2, mu_c2, cov_c2 = m_step(train[:, 500:], alpha_c2, mu_c2, cov_c2, expect_c2, J)
#         if i == 199:
#             print('em iteration i =', i)
#             for k in range(J):
#                 print('alpha_c2', k, '=', alpha_c2[k])
#             for k in range(J):
#                 print('mu_c2', k, '=', mu_c2[:, k])
#             for k in range(J):
#                 print('cov_c2', k, '=\n', cov_c2[:, :, k])

    #### test
    # calculate likehood correspond to class 1 for each data in the test set
    value_c1 = np.zeros(1000)
    for i in range(J):
        value_c1 = value_c1 + alpha_c1[i] * pdf(np.transpose(test), mu_c1[:, i], cov_c1[:, :, i])

    # calculate likehood correspond to class 1 for each data in the test set
    value_c2 = np.zeros(1000)
    for i in range(J):
        value_c2 = value_c2 + alpha_c2[i] * pdf(np.transpose(test), mu_c2[:, i], cov_c2[:, :, i])

    # because the priors of the two classes are the same (500/1000, observed from the training data), 
    # so we can simply compare the likelihoods of the two classes while classifying
    pred_as_c1 = value_c1 >= value_c2

    print('J =', J, ', accuracy =', (sum(pred_as_c1[:500]) + sum(~pred_as_c1[500:])) / 1000)
#     print('----------------------------------------------------------------------')

J = 1 , accuracy = 0.852
J = 2 , accuracy = 0.902
J = 3 , accuracy = 0.913
J = 4 , accuracy = 0.916
J = 5 , accuracy = 0.913
J = 6 , accuracy = 0.917
J = 7 , accuracy = 0.911
J = 8 , accuracy = 0.908
J = 9 , accuracy = 0.911


* The test accuracy with J = 1 is much lower than others (i.e. the classification error is much higher than others).
* The models estimated with J = 6 has the highest test accuracy (lowest classification error).